# OCEAN Features: Interpretability & Robustness Analysis



In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap
import warnings
warnings.filterwarnings('ignore')

from text_features.personality import OCEAN_DIMS

# Set plotting style
sns.set_style('whitegrid')
shap.initjs()  # Initialize SHAP JS visualizations

## 1. Load Trained Model & Data

In [ ]:
# Load the trained XGBoost + OCEAN model from previous notebook
model_path = '../artifacts/xgb_ocean_model.joblib'
model = joblib.load(model_path)

print(f"Model loaded from {model_path}")
print(f"Pipeline steps: {model.named_steps.keys()}")

In [ ]:
# Load test data (you may need to regenerate from notebook 03)
# For demo purposes, we'll load a sample

# TODO: Replace this with actual X_test, y_test from notebook 03
# For now, create placeholder
print("Note: Load X_test, y_test from notebook 03 or regenerate data")

## 2. Feature Importance Analysis

In [ ]:
# Extract feature names after preprocessing
feature_names = model.named_steps['preprocess'].get_feature_names_out()

# Get feature importances from XGBoost
importances = model.named_steps['clf'].feature_importances_

# Create DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("Top 20 Features by Importance:")
print(importance_df.head(20))

In [ ]:
# Visualize top 30 features
top_features = importance_df.head(30)

plt.figure(figsize=(10, 12))
plt.barh(range(len(top_features)), top_features['importance'], color='steelblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance (Gain)')
plt.title('Top 30 XGBoost Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../artifacts/results/feature_importance_top30.png', dpi=150)
plt.show()

In [ ]:
# Check ranking of OCEAN features
ocean_importance = importance_df[importance_df['feature'].str.contains('|'.join(OCEAN_DIMS), case=False)]

print("\nOCEAN Features Importance Ranking:")
print(ocean_importance)

if len(ocean_importance) > 0:
    avg_rank = importance_df.reset_index(drop=True).reset_index().merge(
        ocean_importance, on='feature'
    )['index'].mean()
    print(f"\nAverage rank of OCEAN features: {avg_rank:.1f} / {len(importance_df)}")

## 3. SHAP Analysis

### 3.1 Global Feature Importance

In [ ]:
# Transform test data through preprocessing pipeline
# X_test_transformed = model.named_steps['preprocess'].transform(X_test)

# Create SHAP explainer
# explainer = shap.TreeExplainer(model.named_steps['clf'])
# shap_values = explainer(X_test_transformed)

print("TODO: Uncomment and run after loading X_test from notebook 03")

In [ ]:
# Global SHAP summary plot (beeswarm)
# shap.summary_plot(shap_values, X_test_transformed, 
#                  feature_names=feature_names, max_display=20)
# plt.savefig('../artifacts/results/shap_summary_beeswarm.png', dpi=150, bbox_inches='tight')

In [ ]:
# SHAP bar plot (mean absolute SHAP values)
# shap.summary_plot(shap_values, X_test_transformed, 
#                  feature_names=feature_names, plot_type='bar', max_display=20)
# plt.savefig('../artifacts/results/shap_summary_bar.png', dpi=150, bbox_inches='tight')

### 3.2 OCEAN-Specific SHAP Analysis

In [ ]:
# Extract SHAP values for OCEAN dimensions only
# ocean_indices = [i for i, name in enumerate(feature_names) 
#                 if any(dim in name for dim in OCEAN_DIMS)]

# if ocean_indices:
#     ocean_feature_names = [feature_names[i] for i in ocean_indices]
#     ocean_shap_values = shap_values.values[:, ocean_indices]
#     ocean_features = X_test_transformed[:, ocean_indices]
#     
#     plt.figure(figsize=(10, 6))
#     shap.summary_plot(ocean_shap_values, ocean_features, 
#                      feature_names=ocean_feature_names)
#     plt.title('SHAP Values for OCEAN Dimensions')
#     plt.savefig('../artifacts/results/shap_ocean_only.png', dpi=150, bbox_inches='tight')
#     plt.show()

### 3.3 Local Explanations (Individual Cases)

In [ ]:
# Select interesting samples:
# - High-risk prediction (model says likely default)
# - Low-risk prediction (model says unlikely default)

# y_proba = model.predict_proba(X_test)[:, 1]
# high_risk_idx = np.argmax(y_proba)
# low_risk_idx = np.argmin(y_proba)

# print(f"High-risk sample index: {high_risk_idx} (predicted prob: {y_proba[high_risk_idx]:.3f})")
# print(f"Low-risk sample index: {low_risk_idx} (predicted prob: {y_proba[low_risk_idx]:.3f})")

In [ ]:
# Force plot for high-risk sample
# shap.force_plot(explainer.expected_value, 
#                shap_values.values[high_risk_idx], 
#                X_test_transformed[high_risk_idx],
#                feature_names=feature_names)

In [ ]:
# Force plot for low-risk sample
# shap.force_plot(explainer.expected_value, 
#                shap_values.values[low_risk_idx], 
#                X_test_transformed[low_risk_idx],
#                feature_names=feature_names)

## 4. Correlation Analysis

Check if OCEAN features are redundant with existing features

In [ ]:
# Compute correlations between OCEAN and key numeric features
# key_features = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'revol_util'] + OCEAN_DIMS
# corr_matrix = X_train[key_features].corr()

# plt.figure(figsize=(12, 10))
# sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
#            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
# plt.title('Correlation: OCEAN vs Key Financial Features')
# plt.tight_layout()
# plt.savefig('../artifacts/results/ocean_correlation_with_features.png', dpi=150)
# plt.show()

In [ ]:
# Check maximum correlation of each OCEAN dimension with other features
# for dim in OCEAN_DIMS:
#     corr_with_others = corr_matrix[dim].drop(OCEAN_DIMS).abs().sort_values(ascending=False)
#     print(f"\n{dim.capitalize()}:")
#     print(f"  Highest correlation: {corr_with_others.index[0]} ({corr_with_others.iloc[0]:.3f})")
#     print(f"  Top 3: {corr_with_others.head(3).to_dict()}")

## 5. Robustness Checks

### 5.1 Text Truncation Sensitivity

Test how different truncation lengths affect OCEAN scores and model performance

In [ ]:
# Test different max_chars: [400, 800, 1200]
# For offline mode, this will show consistency of deterministic fallback
# For API mode, this tests prompt sensitivity

from text_features.personality import OceanScorer

# Sample 100 texts
# sample_titles = X_train['title_clean'].sample(100, random_state=42).tolist()
# sample_emp = X_train['emp_title_clean'].sample(100, random_state=42).tolist()

# truncation_lengths = [400, 800, 1200]
# truncation_results = {}

# for max_chars in truncation_lengths:
#     scorer = OceanScorer(offline_mode=True, max_chars=max_chars)
#     scores = scorer.score_batch(sample_titles, sample_emp)
#     scores_df = pd.DataFrame(scores)
#     truncation_results[max_chars] = scores_df
#     print(f"Max chars {max_chars}: Mean OCEAN scores")
#     print(scores_df.mean())
#     print()

### 5.2 Consistency Check (Repeated Scoring)

For offline mode: should be 100% consistent (deterministic)
For API mode with temperature=0: should have very low variance

In [ ]:
# Score the same 10 samples multiple times
# sample_text = "Debt Consolidation | Software Engineer"
# 
# scorer = OceanScorer(offline_mode=True)
# repeated_scores = []
# 
# for i in range(5):
#     score = scorer.score("Debt Consolidation", "Software Engineer")
#     repeated_scores.append(score)
# 
# repeated_df = pd.DataFrame(repeated_scores)
# print("Repeated scoring consistency (std should be 0 for offline mode):")
# print(repeated_df.std())

## 6. Summary & Insights

### Key Findings

**Feature Importance:**
- OCEAN features rank: _____ (fill after analysis)
- Most important OCEAN dimension: _____
- Least important OCEAN dimension: _____

**SHAP Insights:**
- Direction of effects: Does high conscientiousness reduce default risk? _____
- Magnitude: How large are OCEAN effects compared to financial features? _____

**Correlation:**
- Max correlation with existing features: _____ (should be < 0.7 for independence)
- Evidence of multicollinearity: _____

**Robustness:**
- Truncation sensitivity: _____
- Scoring consistency: _____

### Recommendations
1. If OCEAN features show reasonable importance + interpretable directions → **Production candidate**
2. If highly correlated with existing features → **May be redundant**
3. If unstable across prompts/truncation → **Need better text or prompt engineering**